In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
f = open("data/lichess_db_standard_rated_2018-01.pgn")

In [20]:
def get_next_analysed_game():
    # Пока не найдем игру с анализом
    while True:
        # Записываем строки игры
        buffer = []
        while True:
            line = f.readline()

            # Пустые строки не нужны
            if line == "\n":
                continue

            buffer.append(line)

            # Ходы идут последней строкой в игре
            # и всегда начинаются с "1."
            if line.startswith("1."):
                break
        
        if ("%eval" in line):
            return buffer
        
def params_to_dict(str_list):
    """
    Принимает строки в виде
    [key "value"]
    И переводит их в словарь
    https://en.wikipedia.org/wiki/Portable_Game_Notation
    """
    return {
        a: b.strip('"') 
        for a, b in [
            i.strip("\n").strip("[]").split(" ", 1) 
            for i in str_list
        ]
    }

def moves_to_df(moves):
    s = moves.replace("[", "").replace("]", "")
    s = s.split(" ")
    s = s[:-1]
    
    # Если последний ход - мат, для него нет оценки позиции (игра окончена)
    # Поэтому слайсы сдвигаются, время не в той колонке
    # Чтобы исправить, чуть подправим в конце
    if len(s) % 8 != 0:
        s.append(s[-2])
        s[-3] = "Checkmate"

    df = pd.DataFrame.from_dict({
                
        "WhiteMove": s[1::16],
        "WhiteEval": s[4::16],
        "WhiteTime": s[6::16],
        
        "BlackMove": s[9::16],
        "BlackEval": s[12::16],
        "BlackTime": s[14::16],
    
    }, orient="index").transpose()
    
    df["MoveNumber"] = df.index + 1
    
    # Анализ доступен только для первых 100 ходов
    # Строчки без него не нужны
    df = df.head(100)

    return df
    
def get_and_parse_next_analysed_game():
    game = get_next_analysed_game()
    return (
        params_to_dict(game[:-1]), 
        moves_to_df(game[-1])
    )

In [39]:
a, m = get_and_parse_next_analysed_game()

In [40]:
a

{'Event': 'Rated Bullet game',
 'Site': 'https://lichess.org/uJquHgXZ',
 'White': 'twotothepowersix',
 'Black': 'MM2010',
 'Result': '1-0',
 'UTCDate': '2018.01.01',
 'UTCTime': '08:29:52',
 'WhiteElo': '1892',
 'BlackElo': '1733',
 'WhiteRatingDiff': '+6',
 'BlackRatingDiff': '-6',
 'ECO': 'C50',
 'Opening': 'Italian Game: Rousseau Gambit',
 'TimeControl': '120+0',
 'Termination': 'Normal'}

In [41]:
m

,WhiteMove,WhiteEval,WhiteTime,BlackMove,BlackEval,BlackTime,MoveNumber
0,e4,0.09,0:02:00,e5,0.23,0:02:00,1
1,Nf3,0.25,0:01:59,Nc6,0.26,0:02:00,2
2,Bc4,0.17,0:01:57,f5?!,0.94,0:02:00,3
3,Nc3?,-1.75,0:01:53,Nf6?,0.68,0:02:00,4
4,d3,0.23,0:01:52,fxe4,0.32,0:01:59,5
5,dxe4,0.17,0:01:52,Bc5?,1.32,0:01:57,6
6,O-O,1.02,0:01:50,a6?,2.43,0:01:53,7
7,Bg5?,1.25,0:01:49,b5,1.55,0:01:53,8
8,Bb3,1.43,0:01:43,b4?,4.25,0:01:52,9
9,Nd5,3.84,0:01:41,a5??,8.14,0:01:45,10


In [7]:
for i in range(10_000):
    a, m = get_and_parse_next_analysed_game()
    if int(a["WhiteElo"]) < 800:
        print(a["Site"])

https://lichess.org/mKgJeZYp
https://lichess.org/KlEeoWq2
https://lichess.org/Htg4futi
https://lichess.org/HcIKjFqM
https://lichess.org/6HuZD1th
https://lichess.org/kHcu59Ug
https://lichess.org/J7rWmBG2


In [ ]:
## Notes

In [ ]:
# Может быть неопределенный исход
# [Result "*"]
# Их лучше исключить из анализа

In [ ]:
# for i in range(10_000):
#     g = get_and_parse_next_analysed_game()
#     if int(g["WhiteElo"]) < 800:
#         print(g["Site"])